In [1]:
import sys
sys.path.append('.')

import numpy as np

from annotation_ems.gen_dataset import generate_dataset
from annotation_ems.ems_prepare import split, prepare_json
import itertools as it

import subprocess
import os
import time
import random

### begin of config

dataset_path = '/mnt/data/yxchen/gesture-datasets/ems'
output_path = './annotation_ems/'

expr_name_train = ""
expr_name_test = ""
modality = "rgb"  # d, rgb, rgbd
labels = ['wrist_left', 'wrist_right', 'pronation', 'supination']

In [2]:
data_batch = ['subject01_machine_recovery_3gps_2pairs', \
             'subject01_machine_recovery_3gps_2pairs_03', \
             'subject01_machine_recovery_3gps_2pairs_04', \
             'subject01_machine_recovery_3gps_2pairs_05', \
             'subject01_machine_recovery_3gps_2pairs_06']
#data_batch = [6,5,4,3,2,1]
data_batch.sort()

In [3]:
#instance_ratio = 4
#combinations = list(it.combinations(data_batch, instance_ratio))

In [4]:
#combinations

In [18]:
train_ratio = 50
test_ratio = 50
count = 1
expr_name = "1.1"

train_partition = {x: train_ratio for x in data_batch}
test_partition = {x: test_ratio for x in data_batch}
generate_dataset(expr_name=expr_name, modality=modality, dataset_path=dataset_path, output_path=output_path, train_partition=train_partition, test_partition=test_partition, labels=labels)
prepare_json(csv_dir_path=output_path, expr_name=expr_name)
print(expr_name)

train_ratio_verify = 50
test_ratio_verify = 50
expr_name_ver = "verify"
count = 1
for dataset in data_batch:
    name = '000%d' % count if (count <= 10) else \
    ('00%d' % count if count <=100 else \
    ('0%d' % count if count <= 1000 else \
    ('%d' %count)))
    print(expr_name_ver+name)
    train_partition = {dataset: train_ratio_verify }    
    test_partition = {dataset: test_ratio_verify}
    generate_dataset(expr_name=expr_name_ver+name, modality=modality, dataset_path=dataset_path, output_path=output_path, train_partition=train_partition, test_partition=test_partition, labels=labels)
    prepare_json(csv_dir_path=output_path, expr_name=expr_name_ver+name)
    count += 1

1.1
verify0001
verify0002
verify0003
verify0004
verify0005


In [13]:
count = 1
processes = set()
max_processes = 4

for dataset in data_batch:
    if (count < 5):
        count += 1
        continue
    name = '000%d' % count if (count <= 10) else \
        ('00%d' % count if count <=100 else \
        ('0%d' % count if count <= 1000 else \
        ('%d' %count)))
    cmd = "CUDA_VISIBLE_DEVICES=%d nohup python main.py \
        --root_path /mnt/data/jarvislam1999/Real-time-GesRec \
        --video_path /mnt/data/yxchen/gesture-datasets/ems \
        --annotation_path annotation_ems/ems%s.json\
        --result_path results/ems%s \
        --pretrain_path /mnt/data/yxchen/model-zoo/jester_resnext_101_RGB_32.pth \
        --dataset ems \
        --sample_duration 32 \
        --learning_rate 0.01 \
        --model resnext \
        --model_depth 101 \
        --resnet_shortcut B \
        --batch_size 12 \
        --n_classes 27 \
        --n_finetune_classes 4 \
        --n_threads 16 \
        --checkpoint 1 \
        --modality RGB \
        --train_crop random \
        --n_val_samples 1 \
        --test_subset test \
        --n_epochs 40 \
        --no_val \
        --checkpoint 5\
    " % ((count-1) % 4,expr_name_ver+name,expr_name_ver+name)
    print(expr_name_ver+name)
    out = open('ems%sout.txt'%(expr_name_ver+name), 'w')
    processes.add(subprocess.Popen(cmd, shell=True, stdout = out, stderr = out))
    #print(processes)
    count += 1
    if len(processes) >= max_processes:
        os.wait()
        processes.difference_update(
            [p for p in processes if p.poll() is not None])

cmd = "CUDA_VISIBLE_DEVICES=%d nohup python main.py \
        --root_path /mnt/data/jarvislam1999/Real-time-GesRec \
        --video_path /mnt/data/yxchen/gesture-datasets/ems \
        --annotation_path annotation_ems/ems%s.json\
        --result_path results/ems%s \
        --pretrain_path /mnt/data/yxchen/model-zoo/jester_resnext_101_RGB_32.pth \
        --dataset ems \
        --sample_duration 32 \
        --learning_rate 0.01 \
        --model resnext \
        --model_depth 101 \
        --resnet_shortcut B \
        --batch_size 12 \
        --n_classes 27 \
        --n_finetune_classes 4 \
        --n_threads 16 \
        --checkpoint 1 \
        --modality RGB \
        --train_crop random \
        --n_val_samples 1 \
        --test_subset test \
        --n_epochs 40 \
        --no_val \
        --checkpoint 5\
    " % ((count-1) % 4,expr_name,expr_name)
print(expr_name)
out = open('ems%sout.txt'%(expr_name), 'w')
#processes.add(subprocess.Popen(cmd, shell=True, stdout = out, stderr = out))
#Check if all the child processes were closed
for p in processes:
    if p.poll() is None:
        p.wait()
        


verify0005
1.1


In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "%d" % random.randrange(0,4)

import sys
# sys.path.append('../')
os.chdir("/mnt/data/jarvislam1999/Real-time-GesRec/")

from ems_tester import EMSTester

In [17]:
accuracy_list = []
count_test = 1
for dataset in data_batch:
    name_test = '000%d' % count_test if (count_test <= 10) else \
        ('00%d' % count_test if count_test <=100 else \
        ('0%d' % count_test if count_test <= 1000 else \
        ('%d' %count_test)))
    print(name_test)
    tester = EMSTester(root_path='/mnt/data/jarvislam1999/Real-time-GesRec',
              video_path='/mnt/data/yxchen/gesture-datasets/ems',
              annotation_path='annotation_ems/ems%s.json' % (expr_name_ver+name_test),
              result_path='results/ems%s_test' % (expr_name_ver+name_test),
              model_path='results/ems%s/save_30.pth' %(expr_name_ver+name_test))
    y_pred, y_true, test_data = tester.test()
    accuracy_list.append(np.sum(np.equal(y_pred, y_true))/len(y_pred)*100)
    count_test+=1


0001
Namespace(annotation_path='/mnt/data/jarvislam1999/Real-time-GesRec/annotation_ems/emsverify0001.json', arch='resnext-101', batch_size=1, begin_epoch=1, checkpoint=1, crop_position_in_test='c', dampening=0.9, dataset='ems', ft_begin_index=0, initial_scale=1.0, learning_rate=0.1, lr_patience=10, lr_steps=[10, 25, 50, 80, 100], manual_seed=1, mean=[114.7748, 107.7354, 99.475], mean_dataset='activitynet', modality='RGB', model='resnext', model_depth=101, momentum=0.9, n_classes=400, n_epochs=200, n_finetune_classes=4, n_scales=5, n_threads=1, n_val_samples=1, nesterov=False, no_cuda=False, no_hflip=False, no_mean_norm=False, no_softmax_in_test=False, no_train=False, no_val=False, norm_value=1, optimizer='sgd', pretrain_path='', resnet_shortcut='B', resnext_cardinality=32, result_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/emsverify0001_test', resume_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/emsverify0001/save_30.pth', root_path='/mnt/data/jarvislam1999/Real-ti

[INFO]: RGB model is used for init model
Total number of trainable parameters:  47529412
[INFO]: EMS Dataset - testing is loading...
dataset loading [0/200]
loading checkpoint /mnt/data/jarvislam1999/Real-time-GesRec/results/emsverify0005/save_30.pth
Batch time: 0
Batch time: 0.15130516529083252
-----Evaluation is finished------
Overall Prec@1 99.00000%
Namespace(annotation_path='/mnt/data/jarvislam1999/Real-time-GesRec/annotation_ems/ems1.1.json', arch='resnext-101', batch_size=1, begin_epoch=1, checkpoint=1, crop_position_in_test='c', dampening=0.9, dataset='ems', ft_begin_index=0, initial_scale=1.0, learning_rate=0.1, lr_patience=10, lr_steps=[10, 25, 50, 80, 100], manual_seed=1, mean=[114.7748, 107.7354, 99.475], mean_dataset='activitynet', modality='RGB', model='resnext', model_depth=101, momentum=0.9, n_classes=400, n_epochs=200, n_finetune_classes=4, n_scales=5, n_threads=1, n_val_samples=1, nesterov=False, no_cuda=False, no_hflip=False, no_mean_norm=False, no_softmax_in_test=Fa

In [19]:
tester = EMSTester(root_path='/mnt/data/jarvislam1999/Real-time-GesRec',
          video_path='/mnt/data/yxchen/gesture-datasets/ems',
          annotation_path='annotation_ems/ems%s.json' % (expr_name),
          result_path='results/ems%s_test' % (expr_name),
          model_path='results/ems%s/save_30.pth' %(expr_name))
y_pred, y_true, test_data = tester.test()


Namespace(annotation_path='/mnt/data/jarvislam1999/Real-time-GesRec/annotation_ems/ems1.1.json', arch='resnext-101', batch_size=1, begin_epoch=1, checkpoint=1, crop_position_in_test='c', dampening=0.9, dataset='ems', ft_begin_index=0, initial_scale=1.0, learning_rate=0.1, lr_patience=10, lr_steps=[10, 25, 50, 80, 100], manual_seed=1, mean=[114.7748, 107.7354, 99.475], mean_dataset='activitynet', modality='RGB', model='resnext', model_depth=101, momentum=0.9, n_classes=400, n_epochs=200, n_finetune_classes=4, n_scales=5, n_threads=1, n_val_samples=1, nesterov=False, no_cuda=False, no_hflip=False, no_mean_norm=False, no_softmax_in_test=False, no_train=False, no_val=False, norm_value=1, optimizer='sgd', pretrain_path='', resnet_shortcut='B', resnext_cardinality=32, result_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/ems1.1_test', resume_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/ems1.1/save_30.pth', root_path='/mnt/data/jarvislam1999/Real-time-GesRec', sample_duratio

In [20]:
accuracy_list

[99.494949494949495, 99.5, 99.0, 98.5, 99.0]